In [4]:
import mido
from mido import MidiFile, MidiTrack, Message

def create_detuned_cmaj7_mpe(filename):
    # Create a new MIDI file and a new track
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Constants for note durations and volumes
    duration = 480  # duration in ticks (e.g., quarter note in standard MIDI)
    volume = 64  # Volume (0-127)

    # Define the MIDI note numbers for Cmaj7 chord
    C_note = 60  # Middle C
    E_note = 64  # E above middle C
    G_note = 67  # G above middle C
    B_note = 71  # B above middle C

    # MIDI channels for MPE (channels 1-15 for notes, channel 0 is reserved for global control)
    C_channel = 1
    E_channel = 2
    G_channel = 3
    B_channel = 4

    # Function to create pitch bend messages
    def create_pitch_bend(value, channel):
        # Pitch bend value ranges from -8192 to 8191
        pitch = int((value / 100) * 8192)
        return Message('pitchwheel', pitch=pitch, channel=channel)

    # Function to set pitch bend sensitivity
    def set_pitch_bend_sensitivity(sensitivity, channel):
        msb = sensitivity
        lsb = 0  # No microtonal fractions
        return [
            Message('control_change', control=101, value=0, channel=channel),  # RPN MSB
            Message('control_change', control=100, value=0, channel=channel),  # RPN LSB
            Message('control_change', control=6, value=msb, channel=channel),  # Data Entry MSB
            Message('control_change', control=38, value=lsb, channel=channel)  # Data Entry LSB
        ]

    # Set pitch bend sensitivity to ±48 semitones for all channels
    for channel in [C_channel, E_channel, G_channel, B_channel]:
        track.extend(set_pitch_bend_sensitivity(48, channel))

    # Time for the start of the chord
    start_time = 0

    # Add the C note to the MIDI file (no detuning)
    track.append(Message('note_on', note=C_note, velocity=volume, channel=C_channel, time=start_time))

    # Add the detuned E note (27 cents down)
    track.append(create_pitch_bend(-27, E_channel))  # -27 cents detune
    track.append(Message('note_on', note=E_note, velocity=volume, channel=E_channel, time=start_time))

    # Add the G note to the MIDI file (no detuning)
    track.append(Message('note_on', note=G_note, velocity=volume, channel=G_channel, time=start_time))

    # Add the B note to the MIDI file (no detuning)
    track.append(Message('note_on', note=B_note, velocity=volume, channel=B_channel, time=start_time))

    # Add note off messages for all notes at the same time (duration ticks later)
    track.append(Message('note_off', note=C_note, velocity=volume, channel=C_channel, time=duration))
    track.append(Message('note_off', note=E_note, velocity=volume, channel=E_channel, time=0))  # time=0 because it’s the same moment
    track.append(Message('note_off', note=G_note, velocity=volume, channel=G_channel, time=0))
    track.append(Message('note_off', note=B_note, velocity=volume, channel=B_channel, time=0))
    #track.append(create_pitch_bend(0, E_channel))  # Reset pitch bend for E

    # Save the MIDI file
    mid.save(filename)
    print(f"MIDI file generated: {filename}")

# Call the function to create the detuned Cmaj7 chord MIDI file
create_detuned_cmaj7_mpe('detuned_Cmaj7_chord_mpe.mid')


MIDI file generated: detuned_Cmaj7_chord_mpe.mid
